# Watershed transform (digital image segmentation)

In [1]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

## Image segmentation

In digital image processing and computer vision, image segmentation is the process of partitioning a digital image into multiple image segments, also known as image regions or image objects (sets of pixels). The goal of segmentation is to simplify and/or change the representation of an image into something that is more meaningful and easier to analyze. Image segmentation is typically used to locate objects and boundaries (lines, curves, etc.) in images. More precisely, image segmentation is the process of assigning a label to every pixel in an image such that pixels with the same label share certain characteristics.

The result of image segmentation is a set of segments that collectively cover the entire image, or a set of contours extracted from the image (edge detection). Each of the pixels in a region are similar with respect to some characteristic or computed property, such as color, intensity, or texture. Adjacent regions are significantly different with respect to the same characteristic(s). When applied to a stack of images, typical in medical imaging, the resulting contours after image segmentation can be used to create 3D reconstructions with the help of geometry reconstruction algorithms like marching cubes.

## Task definition and mathematical model

As stated above we are trying to find the partitioning of a digital image into multiple regions (sets of pixels).

Let $R$ represent the entire spatial region occupied by an image. We may view image segmantation as a process that partitions $R$ into $n$ subregions, $R_1$, $R_2$,..., $R_n$ such that:

(a) $\bigcup_{i=1}^nR_i=R$ $\iff$ the segmentation must be complete - every pixel must be in a region

(b) $R_i$ is a connceted set, for $i = 0, 1, 2, ..., n$ $\iff$ points in a region must be connected in a predefined sense (e.g., 8-connected)

(c) $R_{i} \cap R_{j} = \varnothing$, for all i and j, $i \neq j$ $\iff$ different regions must be $\textit{disjoint}$

(d) $Q(R_i) = \textnormal{TRUE}$, for $i = 0, 1, 2, ..., n$ $\iff$ pixels in a region must statisfy some property defined by prediacte $Q$ (e.g., intensity)

(e) $Q(R_{i} \cup R_{j}) = \textnormal{FALSE}$, for any $\textit{adjacent}$ regions $R_i$ and $R_j$ $\iff$ $\textit{adjacent}$ regions must be different in the sense of predicate $Q$,

where $Q(R_k)$ is a logical predicate defined over the points in set $R_k$ and $\varnothing$ is the null set. The symbols $\cup$ and $\cap$ represent set union and intersection, respectively. Two regions $R_i$ and $R_j$ are said to be $\textit{adjacent}$ if their union forms a connected set. If the set formed by the union of two regions is not connected, the regions are said to be $\textit{disjoint}$.

Thus, we see that the fundamental problem in segmentation is to partition an
image into regions that satisfy conditions (a) — (e). Segmentation algorithms
for monochrome images generally are based on one of two basic categories dealing
with properties of intensity values: $\textit{discontinuity}$ and $\textit{similarity}$. In the first category,
we assume that boundaries of regions are sufficiently different from each other, and
from the background, to allow boundary detection based on local discontinuities in
intensity. $\textit{Edge-based}$ segmentation is the principal approach used in this category.
$\textit{Region-based}$ segmentation approaches in the second category are based on partitioning an image into regions that are similar according to a set of predefined criteria.

## Optimum Global Thresholding Using Otsu's Method

Thresholding may be viewed as a statistical-decision theory problem whose objective is to minimize the average error incurred in assigning pixels to two or more
groups (also called classes). This problem is known to have an elegant closed-form
solution known as the $\textit{Bayes decision function}$. The solution is
based on only two parameters: the probability density function (PDF) of the intensity levels of each class, and the probability that each class occurs in a given application. Unfortunately, estimating PDFs is not a trivial matter, so the problem usually
is simplified by making workable assumptions about the form of the PDFs, such as
assuming that they are Gaussian functions. Even with simplifications, the process
of implementing solutions using these assumptions can be complex and not always
well-suited for real-time applications.

The approach in the following discussion, called Otsu’s method (Otsu 1979), is
an attractive alternative. The method is optimum in the sense that it maximizes the between-class variance, a well-known measure used in statistical discriminant analysis. The basic idea is that properly thresholded classes should be distinct with respect to the intensity values of their pixels and, conversely, that a threshold giving the
best separation between classes in terms of their intensity values would be the best
(optimum) threshold. In addition to its optimality, Otsu’s method has the important
property that it is based entirely on computations performed on the histogram of an
image, an easily obtainable 1-D array.

Otsu's method, named after Nobuyuki Otsu (大津展之, Ōtsu Nobuyuki), is used to perform automatic image thresholding. In the simplest form, the algorithm returns a single intensity threshold that separate pixels into two classes, foreground and background. This threshold is determined by minimizing intra-class intensity variance, or equivalently, by maximizing inter-class variance. Otsu's method is a one-dimensional discrete analogue of Fisher's discriminant analysis, is related to Jenks optimization method, and is equivalent to a globally optimal k-means performed on the intensity histogram.

<img src="understanding_otsu/screenshot_histogram_otsu.png"/>

The algorithm exhaustively searches for the threshold that minimizes the intra-class variance, defined as a weighted sum of variances of the two classes:

$\sigma_{w}^2(t) = \omega_{0}(t)\sigma_{0}^2(t) + \omega_{1}(t)\sigma_{1}^2(t)$

Weights $\omega_0$ and $\omega_1$ are the probabilities of the two classes seperated by a threshold $t$ and $\sigma_{0}^2$ amd $\sigma_{1}^2$ are variances of the two classes.

The class probablitiy $\omega_{\{0, 1\}}(t)$ is computed from the $L$ bins of the histogram:

$\omega_{0}(t) = \sum_{i=0}^{t-1}p(i)$,  $\omega_{1}(t) = \sum_{i=t}^{L-1}p(i)$

For 2 classes, minimizing the intra-class variance is equivalent to maximizing inter-class variance:

$\sigma_{b}^2(t) = \sigma^2 - \sigma_{w}^2(t) = \omega_{0}(t)(\mu_{0}-\mu_{T}^2) + \omega_{1}(t)(\mu_{1}-\mu_{T}^2) = \omega_{0}(t)\omega_{1}(t)[\mu_{0}(t)-\mu_{1}(t)]^2$

which is expressed in terms of class probablities $\omega$ and class means $\mu$, where: 

$\mu_{0}(t) = \frac{\sum_{i=0}^{t-1}ip(i)}{\omega_{0}(t)}$, $\mu_{1}(t) = \frac{\sum_{i=t}^{L-1}ip(i)}{\omega_{1}(t)}$, $\mu_{T} = \sum_{i=0}^{L-1}ip(i)$

The following relations can easily be verified:

$\omega_{0}\mu_{0} + \omega_{1}\mu_{1} = \mu_{T}$

$\omega_{0} + \omega_{1} = 1$ 

The class probabilities and class means can be computed iteratively. This idea yields an effective algorithm:

(1) Compute histogram and pobabilities of each intensity level

(2) Set up initial $\omega_{i}(0)$ and $\mu_{i}(0)$

(3) For all possible thresholds $t = 1, ...,$ $maximum$ $intensity$ do:
        (1) Update $\omega_{i}$ and $\mu_{i}$
        (2) Compute $\sigma_{b}^{2}(t)$

(4) Desired threshold $t^{*}$ corresponds to the maximum $\sigma_{b}^2(t)$

Once $t^{*}$ has been obtained, input image $f(x, y)$ is segmented as follows:

$g(x, y)= 
\begin{cases}
    1,              & f(x, y) > t^{*}\\
    0,              & f(x, y) \leq t^{*}
\end{cases}$

<img src="understanding_otsu/original_f(x,y).jpg"/> $\implies$ <img src="understanding_otsu/binary_g(x,y).jpg"/>

## Watershed

In the study of image processing, a watershed is a transformation defined on a grayscale image. The name refers metaphorically to a geological watershed, or drainage divide, which separates adjacent drainage basins. The watershed transformation treats the image it operates upon like a topographic map, with the brightness of each point representing its height, and finds the lines that run along the tops of ridges.

<img src="understanding_watershed/mathworks_syntethic_dark_blobs.jpg"/> $\iff$ <img src="understanding_watershed/mathworks_syntethic_dark_blobs_topographic_surface.jpg"/>

Any grayscale image can be viewed as a topographic surface where high intensity denotes peaks and hills while low intensity denotes valleys. You start filling every isolated valley (local minima) with different colored water (labels). As the water rises, depending on the peaks (gradients) nearby, water from different valleys with different colors will start to merge. To avoid that, you build barriers (dams) in the locations where water merges. You continue the work of filling water and building barriers until all the peaks are under water. Then the barriers you created give you the segmentation result.

<img src="understanding_watershed/book_dam_construction_via_dilation.jpg"/>

The watershed segmentation working on a grayscale image was introduced by F. Meyer in 1994. During sucessive flooding of the grey value relief, watersheds with adjacent catchment basins are constructed. This flooding process is performed on the gradient image, i.e. the basins should emerge along the edges.

(1) A set of pixels where the flooding should start are chosen. Each is given a different label.

(2) The neighbouring pixels of each marked area are inserted into a priority queue with a priority level corresponding to the gradient magnitude of the pixel.

(3) The pixel with the lowest priority level is extracted from the priority queue. If the neighbours of the extracted pixel that have already been labeled all have the same label, then the pixel is labeled with their label. All non-marked neighbours that are not yet in the priority queue are put into the priority queue.

(4) Repeat (3) until the priority queue is empty.

(5) The non-labeled pixels are the watershed lines.

<img src="understanding_watershed/gradient_flooding.jpg"/>

But this approach gives you oversegmented result due to noise or any other irregularities in the image. Oversegmentation occurs because every regional minimum, even if tiny and insignificant, forms its own catchement basin. Either the image must be pre-processed or the regions must be merged on the basis of a similarity criterion afterwards.

Example of watershed oversegmenation caused by local minima present in the raw unprocessed image of steel grains:

<img src="understanding_watershed/mathworks_steel_grains.jpg"/> $\implies$ <img src="understanding_watershed/mathworks_steel_grains_oversegmentation.jpg"/>

## The use of markers

Direct application of the watershed segmentation algorithm in the form discussed
in the previous section generally leads to oversegmentation, caused by noise and
other local irregularities of the gradient. As the example above illustrates, over-segmentation
can be serious enough to render the result of the algorithm virtually useless. In this
case, this means a large number of segmented regions. A practical solution to this
problem is to limit the number of allowable regions by incorporating a preprocessing stage designed to bring additional knowledge into the segmentation procedure. An approach used to control oversegmentation is based on the concept of markers.

A marker is a connected component belonging to an image. We have internal
markers, associated with objects of interest, and external markers, associated with
the background. A procedure for marker selection typically will consist of two principal steps: 1) preprocessing; and 2) definition of a set of criteria that markers must satisfy. Part of the problem that led to the oversegmented result in the above example is the large number of potential minima. Because of their size, many of these minima are irrelevant detail. An effective method for minimizing the effect of small spatial detail is to filter the image with a smoothing filter.

Suppose that we define an internal marker as 1) a region that is surrounded by
points of higher “altitude”; 2) such that the points in the region form a connected
component; and 3) in which all the points in the connected component have the
same intensity value. The external markers effectively partition the image into regions,
with each region containing a single internal marker and part of the background.
The problem is thus reduced to partitioning each of these regions into two: a single
object, and its background. We can bring to bear on this simplified problem
the segmentation techniques discussed in earlier sections.

OpenCV implemented a marker-based watershed algorithm where you specify which valley points are to be merged and which are not. It is an interactive image segmentation. We give different labels for the objects we know. Label the region which we are sure of being the foreground or object with one color (or intensity), label the region which we are sure of being background or non-object with another color and finally the region which we are not sure of anything, label it with 0. That is our marker. Then apply watershed algorithm. Then our marker will be updated with the labels we gave, and the boundaries of objects will have a value of -1.

## Code

We will first define a general parametric procedure, leveraging the cv2 library, which will allow us to easily experiment with different ways of approaching pre-processing and marker selection:

In [2]:
def watershed_transform(img, blur, dilation):
    #convert to grayscale
    gray = cv2.cvtColor(img, cv.COLOR_BGR2GRAY)
    #find an approximate binary segmentation using Otsu's optimal threshold binarization
    ret, thresh = cv2.threshold(gray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)
    #remove noise using morphological opening
    kernel = np.ones((3,3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv.MORPH_OPEN, kernel, iterations = 2)
    #sure background area
    sure_bg = cv2.dilate(opening, kernel, iterations = 3)
    # find sure foreground area
    dist_transform = cv.distanceTransform(opening, cv.DIST_L2, 5)
    ret, sure_fg = cv.threshold(dist_transform, 0.7*dist_transform.max(), 255, 0)
    #find unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv.subtract(sure_bg, sure_fg)
    #marker labeling
    ret, markers = cv.connectedComponents(sure_fg)
    #add 1 to all labels so that sure background is not 0, but 1
    markers = markers + 1
    #mark the region of unknown with zero
    markers[unknown==255] = 0
    #perform the watershed algorithm which will mark the boundary with -1
    markers = cv.watershed(img, markers)
    #set the boundary pixels to red on the original image
    img[markers == -1] = [255, 0, 0]